In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 19.0 MB/s 
     |████████████████████████████████| 6.6 MB 23.9 MB/s 
     |████████████████████████████████| 596 kB 38.2 MB/s 
     |████████████████████████████████| 101 kB 5.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Spam SNS classification

각 문장이 spam 메일인지 아닌지를 판별하는 문장 분류 태스크  
dataset: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset  
4000개까지 train으로 사용  

Dataset 구축  
dataset에서는 __ getitem __을 통해 한개의 데이터를 모델의 입력형태에 맞추어 반환한다.  
이번 코드에서는 자연어 문장을 모델에 입력하기 위해 tokenization과 vocab dictionary에 따른 index로의 변환을 진행한다.  
또한 label의 ham/spam에 따라 0/1을 label로 변환한다.  

주요 class, method:  
torchtext.data.utils.get_tokenizer: torchtext에서 제공하는 tokenizer 문법에 따른 tokenization을 수행하는 class 반환  
torchtext.vocab.build_vocab_from_iterator:  내 학습데이터에 대한 모든 단어를 입력하면 각 단어에 한개씩 index를 부여한 vocab dictionary 반환  
**huggingface.tokenizer:** 사전에 학습된 tokenization을 불러오는 huggingface class  
https://huggingface.co/docs/transformers/main_classes/tokenizer


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer,BertTokenizer
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

#dataset load
data_path = "/content/drive/MyDrive/랩세미나(new)/tutorial/open_tutorial_DL_NLP/4일차/dataset/spam.csv"
data_df = pd.read_csv(data_path, encoding = "ISO-8859-1")
print(data_df)
print(data_df.columns)

#train test split
data_df = data_df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)
train_df = data_df.loc[:200,:].reset_index()
test_df = data_df.loc[4000:,:].reset_index()
print(train_df)
print(train_df.columns)

#tokenizer load
tokenizer=get_tokenizer("basic_english")
d = data_df.loc[0:3,"v2"]
print(d)
print(tokenizer(data_df.loc[0,"v2"]))

#build vocab dictionary
vocab = build_vocab_from_iterator(list(map(tokenizer,data_df.loc[:,"v2"])))
print(vocab(tokenizer(data_df.loc[0,"v2"])))

#huggingface tokenizer load
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
print(tokenizer(data_df.loc[0,"v2"], max_length=100, padding='max_length', truncation=True))

class myDataset(Dataset):
  def __init__(self, df, tokenizer, vocab) -> None:
      super().__init__()
      self.df = df
      self.tokenizer = tokenizer
      self.vocab = vocab

  def __len__(self):
      return len(self.df)
  
  def __getitem__(self, index):
      data = self.df.loc[index, "v2"]
      target = self.df.loc[index, "v1"]
      
      #data tokenization
      data = self.tokenizer(data, max_length=100,padding='max_length', truncation=True)["input_ids"]

      #labeling
      if target == "ham":
          label = 0
      elif target == "spam":
          label = 1

      return data, label


train_dataset = myDataset(train_df, tokenizer, vocab)
test_dataset = myDataset(test_df, tokenizer, vocab)
batch_size = 100
train_dataloader = DataLoader(train_dataset, batch_size = batch_size)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size)

for i in train_dataset:
  print(i)
  break
  
for i in train_dataloader:
  print(i)
  break

SyntaxError: ignored

model 선언  
문장을 RNN/LSTM을 통해 classification하는 model 선언
1. token의 index를 입력으로 받고 word embedding을 결과로 반환하는 nn.Embedding 사용  
2. LSTM을 통해 모든 token을 입력  
3. many-to-one구조를 가지기 때문에 LSTM의 결과중 마지막 cell에 대한 결과만을 사용하여 nn.linear를통해 classification

입력으로는 문장을 tokenziation과 indexing한게 입력으로 들어오기 때문에(batch, sequance length)형태의 입력  
이후 nn.Embedding을 거치면서 각 단어의 벡터가 생성되기 때문에(batch, sequance length, hidden size)의 형태 사용

주요 obejct:  
**nn.Embedding:**https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html  
**nn.LSTM:**https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html  
**nn.RNN:**https://pytorch.org/docs/stable/generated/torch.nn.RNN.html

In [ ]:
from torch import nn

class myModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.emb = nn.Embedding(50000,128)
        self.lstm = nn.LSTM(128, 128)
        self.linaer = nn.Linear(128, 2)

    def forward(self, x):

        x = self.emb(x)
        x, _ = self.lstm(x)
        
        #마지막 time(seq length)에 대한 결과만을 사용하여 classification
        x = self.linaer(x[:,-1,:])

        return x

model = myModel()

for i in train_dataloader:
  data = i[0]
  label = i[1]
  data = model(data)
  print(data)
  break

tensor([[-0.1231,  0.0153],
        [-0.1830, -0.0108],
        [-0.2121, -0.0281],
        [-0.2269, -0.0392],
        [-0.2350, -0.0463],
        [-0.2394, -0.0507],
        [-0.2420, -0.0536],
        [-0.2433, -0.0554],
        [-0.2441, -0.0566],
        [-0.2444, -0.0573],
        [-0.2445, -0.0578],
        [-0.2445, -0.0581],
        [-0.2445, -0.0582],
        [-0.2443, -0.0583],
        [-0.2442, -0.0584],
        [-0.2441, -0.0584],
        [-0.2440, -0.0584],
        [-0.2439, -0.0584],
        [-0.2438, -0.0583],
        [-0.2437, -0.0583],
        [-0.2436, -0.0583],
        [-0.2435, -0.0583],
        [-0.2435, -0.0582],
        [-0.2434, -0.0582],
        [-0.2434, -0.0582],
        [-0.2434, -0.0582],
        [-0.2433, -0.0582],
        [-0.2433, -0.0582],
        [-0.2433, -0.0581],
        [-0.2432, -0.0581],
        [-0.2432, -0.0581],
        [-0.2432, -0.0581],
        [-0.2432, -0.0581],
        [-0.2432, -0.0581],
        [-0.2432, -0.0581],
        [-0.2432, -0

model train

In [ ]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

model = myModel()
model.cuda()

#학습을 위한 optimizer와 loss function 설정
optimizer = Adam(model.parameters(), lr=0.001)
lf = CrossEntropyLoss().cuda()


#100번의 에폭을 실행
for e in range(100):
  print("\n\nepoch ", e)
  epoch_loss = 0
  train_correct = 0 
  
  #선언한 모델 오브젝트를 학습가능한 상태로 변경
  model.train()

  #모든 학습데이터에 대해서 학습
  for i in train_dataloader:
    #매 배치에 대한 gradient계산 이전에 optimizer에 저장된 이전 batch에 gradient를 삭제(초기화)
    optimizer.zero_grad()
    data = i[0]
    data = data.cuda()
    target = i[1]
    target = target.cuda()

    #결과 도출 및 정답수 연산
    output = model(data)
    pred_label = torch.argmax(output, dim=-1)
    train_correct += sum(pred_label == target.reshape(-1))

    target = target.reshape(-1)
    #loss연산
    loss = lf(output, target)
    #print(loss)

    #loss backpropagation
    loss.backward()

    #gradient update
    optimizer.step()

    epoch_loss += loss.item()
  
  print(train_correct)
  print("train loss", epoch_loss/len(train_dataloader))
  print("train acc", train_correct/len(train_dataset))

  #model이 학습되지 않는 상태로 변경
  model.eval()
  test_loss = 0
  test_correct = 0 

  #gradient를 계산하지 않도록 하여 cost낭비 방지
  with torch.no_grad():
    #모든 test dataset에 대해서 결과연산
    for i in test_dataloader:
      data = i[0]
      target = i[1]
      data = data.cuda()
      target = target.cuda()

      output = model(data)

      loss = lf(output, target.reshape(-1))
      pred_label = torch.argmax(output, dim=-1)
      test_correct += sum(pred_label == target.reshape(-1))
      test_loss += loss.item()

  print("test loss", test_loss/len(test_dataloader))
  print("test acc", test_correct/len(test_dataset))
    




epoch  0
tensor(101, device='cuda:0')
train loss 0.6825631856918335
train acc tensor(0.5025, device='cuda:0')
test loss 0.41661655716598034
test acc tensor(0.8651, device='cuda:0')


epoch  1
tensor(168, device='cuda:0')
train loss 0.4302452703317006
train acc tensor(0.8358, device='cuda:0')
test loss 0.40795064345002174
test acc tensor(0.8651, device='cuda:0')


epoch  2
tensor(168, device='cuda:0')
train loss 0.41582388679186505
train acc tensor(0.8358, device='cuda:0')
test loss 0.4223208408802748
test acc tensor(0.8651, device='cuda:0')


epoch  3
tensor(168, device='cuda:0')
train loss 0.410207857688268
train acc tensor(0.8358, device='cuda:0')
test loss 0.41877398639917374
test acc tensor(0.8651, device='cuda:0')


epoch  4
tensor(168, device='cuda:0')
train loss 0.3971082667509715
train acc tensor(0.8358, device='cuda:0')
test loss 0.4098412301391363
test acc tensor(0.8651, device='cuda:0')


epoch  5
tensor(168, device='cuda:0')
train loss 0.3834702620903651
train acc tensor(